In [1]:
import re
import bs4
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
#Function to convert salary info into an average
def salary_cov(x):
    '''Function to convert salary info from a string into an numerical average
    '''
    if x.isdigit(): return float(x)
    else:
        ranges = [float(i) for i in x.split('-')]
        return np.mean(ranges)

After looking through the postings on the Indeed website, I decided to scrape, using BeautifulSoup, from UK and US cities as these seemed to be the bulk of the job postings and as postings from countries like France and Germany would have to be translated.

- London
- Manchester
- Brighton 
- Birmingham
- Bristol
- Edinburgh
- New York
- San Francisco

Since we were only concerned with postings with the salary information, most of the data points ended up being from UK cities as US cities seemed to not share salary information. 

I used the following job titles to capture all the relevant job postings in the broad spectrum of data science roles: 

- Data Scientist
- Business Intelligence Analyst 
- Data Analyst 
- Quantitative Analyst
- Machine Learning 
- Artificial Intelligence 
- Data Engineer

For all of the postings, I pull out of the following information:
- Company (mostly recruitment companies)
- Job title 
- Salary
- Description (the first few lines as shown on the results page)
- Rating (if any)
- Location (if any, although I made sure to record the city so it was easy to distinguish)


### Scraping UK cities

For each city, using the URL, I iterate through all the pages and results to pull the required information for each posting. I then drop all the duplicates and no salary postings, convert all of the salary info in numerical averages (using regex & a function), fill in any empty location and job title information. 

I ran through these steps for each city seperately, although in practise you could put the city names in a city and set a timer to pause between each city to avoid getting blocked by Indeed. 

In [119]:
company = []
job_title = [] 
salary = [] 
description = [] 
rating = [] 
location = []

titles = ['data+scientist', 'business+intelligence', 'data+analyst', 'quantitative+analyst', 
               'machine+learning', 'artificial+intelligence', 'data+engineer']

for title in titles:
    for x in tqdm(range(0, 1000,10)):
        URL = f'https://www.indeed.co.uk/jobs?q={title}&l=Brighton%2C+East+Sussex&start={x}'
        r = requests.get(URL)
        soup = BeautifulSoup(r.text, 'html.parser')

        for entry in soup.find_all('div', attrs = {'class':'result'}):

            try:
                company.append(entry.find('span', attrs = {'class':'company'}).text.replace('\n', '').strip())
            except:
                company.append(np.nan)

            try: 
                job_title.append(entry.find('a', attrs = {'target':'_blank'}).text.replace('\n', '').strip())
            except:
                job_title.append(np.nan)

            try:
                salary.append(entry.find('span', attrs = {'class':'salaryText'}).text.replace('\n', '').strip())
            except:
                salary.append(np.nan)

            try:
                description.append(entry.find('div', attrs = {'class':'summary'}).text.replace('\n', '').strip())
            except:
                description.append(np.nan)

            try:
                rating.append(entry.find('span', attrs = {'class':'ratingsContent'}).text.replace('\n', '').strip())
            except:
                rating.append(np.nan)
            
            try:
                location.append(entry.find('span', attrs = {'class': 'location'}).text.replace('\n', '').strip())
            except:
                location.append(np.nan)

df= pd.DataFrame({'company': company, 'job_title': job_title, 'salary': salary, 'description': description,
                  'rating': rating, 'location': location})

100%|██████████| 100/100 [00:56<00:00,  1.76it/s]


In [121]:
#Drop all of the duplicated rows
print(df.shape)
df.drop(df[df.duplicated()].index, inplace=True)
print(df.shape)

(7914, 6)
(382, 6)


In [122]:
#Drop all of the null salary rows
df.drop(df[df.salary.isnull()].index, inplace=True)
print(df.shape)

(163, 6)


In [123]:
#Using regex, drop all of the rows with salary info in month, hour, week or day
df.drop(df[df.salary.str.contains(r'(month|hour|week|day)', flags=re.IGNORECASE, regex=True)].index, 
            inplace=True)
print(df.shape)

(145, 6)


In [124]:
#Cleaning up the salary values to extract the numerical info
df.salary = df.salary.str.extract(r'([\d£\-,\s]*)', expand=True)
#Stripping the values of everything but digits and ranges
df.salary = df.salary.str.replace('£', '').str.replace(' ', '').str.replace(',', '')

In [125]:
df.salary.fillna(np.nan, inplace=True)
df[df.salary.isnull()]

,company,job_title,salary,description,rating,location


In [126]:
#Mapping the function all the values
df.salary = df.salary.map(salary_cov)

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 1 to 6393
Data columns (total 6 columns):
company        145 non-null object
job_title      145 non-null object
salary         145 non-null float64
description    145 non-null object
rating         55 non-null object
location       145 non-null object
dtypes: float64(1), object(5)
memory usage: 7.9+ KB


In [128]:
df.location.fillna('Brighton', inplace = True )

In [23]:
df.job_title.fillna('Data Scientist', inplace=True)

I saved the cities in seperate csv files to avoid getting confused between the different cities, which I then combine later on.

In [130]:
df.to_csv('/Users/Sangeetha/GA/salary-dataset/salary-brighton.csv', header = df.columns, index=False)

### Scraping US cities

Repeated the same steps as for UK cities

In [55]:
company = []
job_title = [] 
salary = [] 
description = [] 
rating = [] 
location = []

titles = ['machine+learning', 'artificial+intelligence', 'data+engineer']

for title in titles:
    for x in tqdm(range(0, 3000,10)):
        URL = f'https://www.indeed.com/jobs?q={title}&l=New+York%2C+NY&start={x}'
        r = requests.get(URL)
        soup = BeautifulSoup(r.text, 'html.parser')

        for entry in soup.find_all('div', attrs = {'class':'result'}):

            try:
                company.append(entry.find('span', attrs = {'class':'company'}).text.replace('\n', '').strip())
            except:
                company.append(np.nan)

            try: 
                job_title.append(entry.find('a', attrs = {'target':'_blank'}).text.replace('\n', '').strip())
            except:
                job_title.append(np.nan)

            try:
                salary.append(entry.find('span', attrs = {'class':'salaryText'}).text.replace('\n', '').strip())
            except:
                salary.append(np.nan)

            try:
                description.append(entry.find('div', attrs = {'class':'summary'}).text.replace('\n', '').strip())
            except:
                description.append(np.nan)

            try:
                rating.append(entry.find('span', attrs = {'class':'ratingsContent'}).text.replace('\n', '').strip())
            except:
                rating.append(np.nan)
            
            try:
                location.append(entry.find('span', attrs = {'class': 'location'}).text.replace('\n', '').strip())
            except:
                location.append(np.nan)

df_ny= pd.DataFrame({'company': company, 'job_title': job_title, 'salary': salary, 'description': description,
                  'rating': rating, 'location': location})

100%|██████████| 300/300 [07:25<00:00,  1.48s/it]


In [56]:
df=df_ny.copy()

In [57]:
#Drop all of the duplicated rows
print(df.shape)
df.drop(df[df.duplicated()].index, inplace=True)
print(df.shape)

#Drop all of the null salary rows
df.drop(df[df.salary.isnull()].index, inplace=True)
print(df.shape)

(8989, 6)
(2589, 6)
(87, 6)


In [58]:
#Using regex, drop all of the rows with salary info in month, hour, week or day
df.drop(df[df.salary.str.contains(r'(month|hour|week|day)', flags=re.IGNORECASE, regex=True)].index, 
            inplace=True)
print(df.shape)

(62, 6)


In [59]:
#Cleaning up the salary values to extract the numerical info
df.salary = df.salary.str.extract(r'([\d$\-,\s]*)', expand=True)

In [60]:
#Stripping the values of everything but digits and ranges
df.salary = df.salary.str.replace('$', '').str.replace(' ', '').str.replace(',', '')

In [61]:
df.salary.replace('', np.nan, inplace=True)
df.drop(df[df.salary.isnull()].index, inplace=True)

In [62]:
#Mapping the function all the values
df.salary = df.salary.map(salary_cov)

In [63]:
#Convert to GBP, USDGBP = 0.81
df.salary = df.salary*0.81

In [64]:
df

,company,job_title,salary,description,rating,location
34,Columbia University,Research Assistant,40860.450,"Programming skills, with evidence of prior exp...",4.1,"New York, NY"
130,Research Foundation of The City University of ...,Post-doctoral Researcher,51585.660,Development and application of alternatives to...,4.2,"New York, NY"
136,"Converseon, Inc.",Business Development Manager,72900.000,Highly organized and able to interact with pot...,NaN,"New York, NY 10018"
143,Source Enterprises,Cloud Engineer,81000.000,"Automated Platform IaaS deployment, analysis, ...",5.0,"New York, NY"
153,AllocateRite,VP of Partnerships and Business Development,81000.000,NOTE: HIGH-LEVEL WALL STREET CONTACTS ARE A MU...,NaN,"New York, NY 10019"
227,Madison Logic,Data Scientist,105300.000,"Apply your math, optimization, machine learnin...",3.3,"New York, NY 10010"
235,New York City POLICE DEPARTMENT,"City Research Scientist, Level II",68957.325,"Extensive knowledge of applied statistics, ana...",4.2,"New York, NY"
390,RedRoute,Sales Development Representative,38475.000,Our cutting-edge product utilizes AI and Machi...,NaN,"Brooklyn, NY"
626,MetroPlus Health Plan,Senior Data Scientist,87075.000,Apply data mining techniques to create statist...,3.1,"New York, NY"
756,New York City DEPT OF DESIGN & CONSTRUCTION,Project Controls Manager,89882.460,"Candidates with excellent data visualization, ...",3.9,"Queens, NY"


In [69]:
#nyc.to_csv('/Users/Sangeetha/GA/salary-dataset/salary-nyc.csv',header = df.columns, index=False)

### Combining all the dataframes into a single csv file

Combining all of the dataframes containing information about each city in a single file. I double checked the salary, job title and city information was clean and in the correct format before exporting to csv.

In [20]:
london = pd.read_csv('/../../salary-dataset/salary-london.csv')
nyc = pd.read_csv('/../../salary-dataset/salary-nyc.csv')
sf = pd.read_csv('/../../salary-dataset/salary-CA.csv')
birm = pd.read_csv('/../../salary-dataset/salary-birm.csv')
brighton = pd.read_csv('/../../GA/salary-dataset/salary-brighton.csv')
bris = pd.read_csv('/../../salary-dataset/salary-bris.csv')
edin = pd.read_csv('/../../salary-dataset/salary-edin.csv')
manc = pd.read_csv('/../../salary-dataset/salary-manc.csv')

In [21]:
london['city'] = 'London'
nyc['city'] = 'NYC'
sf['city'] = 'San Francisco'
birm['city'] = 'Birmingham'
brighton['city'] = 'Brighton'
bris['city'] = 'Bristol'
edin['city'] = 'Edinburgh'
manc['city'] = 'Manchester'

In [22]:
salary = pd.concat([london, nyc, sf, birm, brighton, bris, edin, manc], ignore_index = True, axis=0)

In [23]:
salary[salary.job_title.isnull()]

,company,job_title,salary,description,rating,location,city
964,Meltwater,NaN,58000.0,Working across a portfolio of 100 existing and...,3.6,London,London
1128,Meltwater,NaN,58000.0,Working across a portfolio of 100 existing and...,NaN,London,London
1290,Meltwater,NaN,58000.0,I'm interested Founded in Norway with just $15...,3.6,London,London
1819,Robert Half United Kingdom,NaN,45000,"Manage, mentor and develop a team of Data anal...",3.9,Birmingham,Birmingham
2004,Robert Half United Kingdom,NaN,29500,Meet the needs of a dynamic business by provid...,3.9,Birmingham,Birmingham
2009,Robert Half United Kingdom,NaN,45000,Match the right techniques and outputs to busi...,3.9,Birmingham,Birmingham
2251,Selby Jennings,NaN,65000,Identification and development of new sales op...,NaN,Brighton,Brighton
2423,BT,NaN,34500,Uses complex data analysis to show underlying ...,3.8,Bristol,Bristol
2431,BT,NaN,34500,Activities carried out have direct impact on b...,3.8,Bristol,Bristol
2497,BT,NaN,34500,Activities carried out have direct impact on b...,NaN,Bristol,Bristol


In [24]:
salary.job_title.fillna('Data Scientist', inplace=True)

In [25]:
salary.shape

(3106, 7)

In [26]:
salary[salary.salary=='salary']

,company,job_title,salary,description,rating,location,city
915,company,job_title,salary,description,rating,location,London
1665,company,job_title,salary,description,rating,location,NYC
1775,company,job_title,salary,description,rating,location,San Francisco


In [27]:
salary.drop(salary[salary.salary=='salary'].index, inplace=True)

In [28]:
salary.shape

(3103, 7)

In [29]:
salary.to_csv('/../../salary-full.csv', header = salary.columns, index=False)

In [4]:
salary.city.value_counts()

London           1511
Birmingham        379
Manchester        378
Bristol           315
NYC               210
Brighton          145
Edinburgh          84
San Francisco      81
Name: city, dtype: int64